## キーワード検索

In [85]:
import elasticsearch, json
import pandas as pd

HOST = "192.168.1.31"
PORT = "9200"
INDEX_NAME = "mlhw"
SCROLL_TIME = '1m'
SCROLL_SIZE = 10
pd.set_option("display.max_colwidth", 100)

search_word = "カバー"

client = elasticsearch.Elasticsearch(HOST + ":" + PORT)

results = []
scroll_results = client.search(
    index = INDEX_NAME,
    scroll = SCROLL_TIME,
    size = SCROLL_SIZE,
    body = {"query": { "match": {"文章":search_word} }}
)
sid = scroll_results['_scroll_id']
scroll_results = scroll_results['hits']['hits']
scroll_size = len(scroll_results)

for scroll_result in scroll_results:
    results.append({"id":scroll_result['_id'], "category": scroll_result['_source']['業種(大分類)'], "sentence":scroll_result['_source']['文章']})

while scroll_size > 0:
    scroll_results = client.scroll(
        scroll_id = sid,
        scroll = SCROLL_TIME )
    sid = scroll_results['_scroll_id']
    scroll_results = scroll_results['hits']['hits']
    scroll_size = len(scroll_results)
    
    for scroll_result in scroll_results:
        results.append({"id":scroll_result['_id'], "category": scroll_result['_source']['業種(大分類)'], "sentence":scroll_result['_source']['文章']})

df = pd.DataFrame(results)
df

,id,category,sentence
0,oabuYW8BVZM1XfUMGlhK,製造業,レールを切断する鋸断機のカバーを開けて鋸断機とカバーの間に立っていたところ、鋸断機横の台車（自動運転）が動き出してカバーを押し、鋸断機とカバーの間に全身をはさまれた状態で発見されたもの。
1,F6buYW8BVZM1XfUMGn34,建設業,杭打ち機のメンテナンス作業中、中間振れ止めカバーが落下し、下部振れ止めカバーとの間に挟まれた。
2,wqbuYW8BVZM1XfUMFy_l,建設業,めっき槽に付属する配管の塗装作業中、めっき槽上に取り付けてあったカバーが外されていたが、被災者はカバーがあると勘違いしてカバーの上を渡ろうとして、めっき槽の中に落ちて被災した。
3,OKbuYW8BVZM1XfUMGl9L,製造業,フェルト製品の製造作業中において、綿を送り出すための装置の下部カバー内で綿が詰まったため、被災者が詰まった綿をカバー下部から取り出していた所、綿が一度に大量に落ち、カバーから引き抜いた際にカバ...
4,yqbuYW8BVZM1XfUMFx7k,建設業,天井クレーン用走行レールの点検作業後、梯子で地上に降りる途中、両手に持っていた車輪カバーのカバーが開いたため体勢が崩れ、約６．３ｍ下の床に転落した。
...,...,...,...
111,vabuYW8BVZM1XfUMGl9L,製造業,被災者がコンクリートパイルの骨となる鉄筋籠を製造する鉄筋籠編成機を運転したところ、スライドベースと呼ばれる溶接機をセットし忘れたのに気付き、同機械の運転を停止しないまま安全カバーを開き、回転し...
112,VqbuYW8BVZM1XfUMG5DR,建設業,被災者は同僚とともに、立体駐車場内で15段目のパレット上で鉄骨などのボルトにさび止めの塗装をする作業を行っていた。その後、同僚が他の作業員を迎えに行くため昇降装置に乗り、地下1階まで下降させ、...
113,2abuYW8BVZM1XfUMGnH4,製造業,横挽き機を使って、長板から箸の長さとなる素地を切り出していた被災者が、丸ノコ盤の入ったボックス内から切りくずが吹き出たので、内部で詰まった切りくずを除去するため丸ノコのスイッチを切った。刃のボ...
114,WqbuYW8BVZM1XfUMHbJL,製造業,客先から７Ｆ（７ＦとＲＦの２ストップ）にて小荷物専用昇降機が停止しているとの故障依頼を受け現地出動したが、現地に到着しているはずの本人（被災者）と連絡が取れないため他の保守員を現地確認に向かわ...


## 類似検索したい文書を指定する

In [107]:
# 特定の行を参照する。0始まり
target_id = 10
target = df.loc[target_id]
target

id                                                                       V6buYW8BVZM1XfUMG47R
category                                                                                  製造業
sentence    ＮＣ施盤の機械の間に詰まった切り屑を機械の裏側から取り除いていた際、同僚が起動スイッチを入れたため刃物部分が動き出し、刃物カバーと配線カバーの間に頭部を挟まれた。
Name: 10, dtype: object

## 類似する文書を検索し、トップ10を表示する

In [109]:
# 対象の文章ベクトルを取得する。

id = target['id']
target_detail = client.search(
    index = INDEX_NAME,
    body = {"query": { "match": {"_id":id} }}
)
target_vector = target_detail['hits']['hits'][0]['_source']['scdv_vector']
target_vector

script_query = {
    "script_score": {
        "query": {"match_all": {}},
        "script": {
            "source": "cosineSimilarity(params.query_vector, doc['scdv_vector']) + 1.0",
            "params": {"query_vector": target_vector}
        }
    }
}

# 類似する文書を検索する。
responses = client.search(
    index=INDEX_NAME,
    body={
        "size": SCROLL_SIZE,
        "query": script_query,
        "_source": {"includes": ["_id", "業種(大分類)", "文章"]}
    }
)
# 検索結果を整形する。1件目は検索対象の自分自身である。
responses = responses['hits']['hits']
similar = []
for response in responses:
    id = response['_id']
    score = response['_score']
    category = response['_source']['業種(大分類)']
    sentence = response['_source']['文章']
    similar.append({'ID':id, 'SCORE':score, "分類":category, "文章":sentence})

sim_pd = pd.DataFrame(similar)

sim_pd

,ID,SCORE,分類,文章
0,V6buYW8BVZM1XfUMG47R,2.000000,製造業,ＮＣ施盤の機械の間に詰まった切り屑を機械の裏側から取り除いていた際、同僚が起動スイッチを入れたため刃物部分が動き出し、刃物カバーと配線カバーの間に頭部を挟まれた。
1,J6buYW8BVZM1XfUMG5HR,1.772498,製造業,火打加工機（丸のこ盤）による加工作業中に、トラブルにより機械が停止した。被災者は機械の中をのぞき込みトラブルの原因となった大鋸屑を取り除こうとした際、何らかの原因により機械が動き出し、送材装置...
2,hKbuYW8BVZM1XfUMHJiW,1.756540,建設業,スタビライザーによる地盤改良中に石が詰まりローターが動かなくなったので、ハンマーで叩いて取り除くため、ローター部に潜り込んで石を取り除いたとき、その反動でローターが少し回転すると同時に石で持ち...
3,VabuYW8BVZM1XfUMGnX4,1.754014,製造業,エアープレスでプラスチック製品を成形加工中、プレスの上型と下型の間についたゴミを取り除く際、頭部を入れたままの状態で起動スイッチを押したため挟まれた。
4,yKbuYW8BVZM1XfUMGD-z,1.750116,製造業,木片を破砕するラインの「フライトコンベヤー」で詰まりが発生したため、被災者と同僚労働者は木屑を取り除いていた。同僚労働者が同コンベヤーの上端の木屑を取り除き、地上に下りて制御盤の起動スイッチを...
5,2abuYW8BVZM1XfUMGnH4,1.749695,製造業,横挽き機を使って、長板から箸の長さとなる素地を切り出していた被災者が、丸ノコ盤の入ったボックス内から切りくずが吹き出たので、内部で詰まった切りくずを除去するため丸ノコのスイッチを切った。刃のボ...
6,cabuYW8BVZM1XfUMGnL4,1.745667,製造業,タレットパンチング機械の屑を取り除くためテーブルの送行範囲に入ったところ、同僚が起動スイッチを入れたため、テーブルと建屋の間に挟まれた。
7,p6buYW8BVZM1XfUMG4TR,1.741895,製造業,柱を加工するプレカット加工機械（自動送給）にて、溜った木くずをエアガンにて取除こうとし、柱材を掴んで移動するアームの走行範囲に身体を入れたため、走行してきたアームと機械に挟まれ被災したものである。
8,oqbuYW8BVZM1XfUMGnH4,1.740078,製造業,オガ製造機に材料（材木）を投入、送入オーバーの為機械を停止。材料を押さえるカバーを上げ、手前に引くも動かず、作業位置反対側へ回り、切削刃、手前確認する為、堆積したオガ粉を取り除き確認しようと皮...
9,GabuYW8BVZM1XfUMHJeW,1.728171,製造業,水産工場内の氷の吸送ホッパーを試運転中にホッパーの中に金属部品が入っているのに気付いてこれを取り除こうとして中に入ったときに、まだ電源が切られていなかったため回転羽根がタイマーにより再起動して...
